In [2]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [3]:
tackles = pd.read_csv(f'{project_dir}tackles.csv')
plays = pd.read_csv(f'{project_dir}plays.csv')
players = pd.read_csv(f"{project_dir}players.csv")

In [ ]:
# Fitler run plays ending in a tackle or out of bounds

end_event = ['tackle','out_of_bounds']

run_plays = [list(z) \
             for z in zip(plays[~plays['playDescription'].str.contains('pass')]['gameId'].to_list(),\
                          plays[~plays['playDescription'].str.contains('pass')]['playId'].to_list())]

# Remove plays with unusable data
run_plays_remove = [[2022091801,63],\
                    [2022101605,2970]\
                    [2022102307, 1505]\
                   ]

[run_plays.remove(x) for x in run_plays_remove]

In [ ]:
#Load tracking data for each week
tracking = pd.DataFrame()
for w in range(1,10):
    print('processing week', w)
    if tracking.empty:
        tracking = pd.read_csv(f"{project_dir}/tracking_week_{w}.csv")
        tracking = tracking.merge(players.loc[:, ['nflId', 'position']], how='left')
        tracking = tracking.merge(tackles,  how='left', left_on=['gameId','playId','nflId'], right_on = ['gameId','playId','nflId'])
        tracking = tracking.merge(plays,how='left',on=['gameId','playId'])
    else:
        temp = pd.read_csv(f"{project_dir}/tracking_week_{w}.csv")
        temp = temp.merge(players.loc[:, ['nflId', 'position']], how='left')
        temp = temp.merge(tackles,  how='left', left_on=['gameId','playId','nflId'], right_on = ['gameId','playId','nflId'])
        week = pd.concat([tracking,temp])
        tracking = tracking.merge(plays,how='left',on=['gameId','playId'])

In [ ]:
# Further refine run play dataset to only include plays over 3 seconds

tracking_gameIds = tracking.gameId.unique().tolist()

for gid in tqdm(tracking_gameIds):
    print('Game ID', gid)
    game = tracking.loc[tracking['gameId']==gid].copy()

    #print('filtering run_plays',end='\r', flush=True)
    game_ = pd.DataFrame()
    for i in tqdm(range(len(run_plays)),leave=False):
        game_ = pd.concat([game_,game[(game['gameId'] == run_plays[i][0]) & (game['playId'] == run_plays[i][1])]])

    run_plays_adj = game_[game_['event'].isin(end_event)]['playId'].unique().tolist()

    # Discard plays under 3 seconds
    under_time = []
    for pid in run_plays_adj:
        if game_[game_['playId']==pid]['frameId'].max() < 30:
            under_time.append(pid)
    [run_plays_adj.remove(x) for x in under_time]

In [ ]:
# Special acknowledgment to Nick Wan for providing example code!
# Function to calibrate coordinates to field of play, refine frame number to correspond to start stop events of a play

def calibrate_coords(df_):

    temp = df_.copy()

    temp.loc[temp['playDirection']=='left', 'x'] = 120-temp.loc[temp['playDirection']=='left', 'x']
    temp.loc[temp['playDirection']=='left', 'y'] = (160/3)-temp.loc[temp['playDirection']=='left', 'y']
    temp.loc[temp['playDirection']=='left', 'dir'] = temp.loc[temp['playDirection']=='left', 'dir']+180
    temp.loc[temp['dir']>360, 'dir'] = temp.loc[temp['dir']>360, 'dir']-360
    temp.loc[temp['playDirection']=='left', 'o'] = temp.loc[temp['playDirection']=='left', 'o']+180
    temp.loc[temp['o']>360, 'o'] = temp.loc[temp['o']>360, 'o']-360

    start_events = ['autoevent_ballsnap', 'ball_snap']
    end_events = ['tackle', 'out_of_bounds']

    temp['is_start'] = 0
    if temp['event'].isin(start_events).sum()==0:
        temp.loc[temp['frameId']==temp['frameId'].min(),'is_start'] = 1
    temp.loc[temp['event'].isin(start_events), 'is_start'] = 1

    temp['is_end'] = 0
    temp.loc[temp['event'].isin(end_events), 'is_end'] = 1
    qwerty = temp[temp['is_end']==1][['playId','frameId','event']].drop_duplicates()

    frame_start, frame_end = temp.loc[temp['is_start']==1, 'frameId'].unique()[0], temp.loc[temp['is_end']==1, 'frameId'].unique()[0]

    temp = temp.loc[temp['frameId'].between(frame_start, frame_end)].copy()

    temp = temp.drop(['is_start', 'is_end'], axis=1)

    return temp

In [ ]:
# Calibrate coordinates and save full dataset with only filtered run plays, in chunks
df = pd.DataFrame()
tracking.sort_values(['gameId','playId','frameId','nflId']).reset_index(drop=True)
new_game = 0
game_csv_list = []
for gid in tqdm(run_plays_adj):
    if new_game % 50 == 0:
        df_clean.to_csv(f'{project_dir}{gid[0]}_{gid[1]}.csv')
        game_csv_list.append([gid[0],gid[1]])
        df_clean = pd.DataFrame()
    temp = tracking[(tracking['gameId']==gid[0])&(tracking['playId']==gid[1])].copy()
    df_clean = pd.concat([df_clean,calibrate_coords(temp)])
    new_game += 1

df_clean = pd.concat([df_clean,calibrate_coords(temp)])
df_clean.to_csv(f'{project_dir}week_final.csv')
pd.DataFrame(game_csv_list,columns=['games','plays']).to_csv(f'{project_dir}game_csv_list.csv',index=False)

In [ ]:
# Load the chunked full 9 week dataset with calibrate coords and filtered run plays
df = pd.DataFrame()
for gid in tqdm(game_csv_list):
    df = pd.concat([df,pd.read_csv(f'{project_dir}{gid[0]}_{gid[1]}.csv')])

df = pd.concat([df,pd.read_csv(f'{project_dir}week_final.csv')])

# Clean up loaded data
df = df.iloc[: , 1:]
df[['gameId','playId','nflId','frameId']] = df[['gameId','playId','nflId','frameId']].astype('Int64')

In [ ]:
# Add tackle and assist stats, include a 'both' column for tackle + assist
df = df[df['displayName']!='football']
df['tackle'] = df['tackle'].fillna(0)
df['assist'] = df['assist'].fillna(0)
df['both'] = df['tackle'] + df['assist']
df = df.sort_values(['gameId','playId','frameId','nflId']).reset_index(drop=True)

In [ ]:
# Add/pivot columns for ballcarrier datapoints needed for BIG Score calculations
df_bc = df.loc[df_clean['nflId']==df_clean['ballCarrierId'], ['nflId','gameId', 'playId', 'frameId', 'x','y', 's','a','dir']].rename(columns={'x':'x_bc','y':'y_bc','s':'s_bc', 'a':'a_bc','dir':'dir_bc'}).copy(deep=True)
df_bc = df_bc.sort_values(['gameId','playId','frameId']).reset_index(drop=True)
df_ = df.merge(df_bc,how='left',on=['gameId','playId','frameId'])
df_ = df_.rename(columns={'nflId_x':'nflId'})
df = df_.copy()

# Filter to include only defensive players
df = df[df['club']==df['defensiveTeam']].copy()
df = df.sort_values(by=['nflId','gameId','playId','frameId'])

In [ ]:
# Calculate distance from each defender to ballcarrier
df['dist_to_bc'] = np.sqrt(((df['x_bc'] - df['x'])**2) + ((df['y_bc'] - df['y'])**2))

In [ ]:
# Calculate angle from defender to ballcarrier
df['y_diff'] = df['y_bc'] - df['y']
df.loc[df['y_diff']<0.01,'y_diff'] == 0.01
df['x_diff'] = df['x_bc'] - df['x']
df['slope'] = df['x_diff'].div(df['y_diff'])
df['angle_to_bc'] = np.arctan(df['slope'])*(180/np.pi)

# Using angle to ballcarrier, calculate 'orientation' difference and weighting magnitude
df['o_diff'] = abs(((df['o'] - df['angle_to_bc'])+180) % 360 - 180)
df['o_mod'] = np.cos(np.deg2rad(df['o_diff']))
df.loc[df['o_mod']<0,'o_mod'] = 0
df.loc[df['angle_to_bc'].isna(),'o_diff'] = 0
df.loc[df['angle_to_bc'].isna(),'o_mod'] = 1

# Using angle to ballcarrier, calculate 'direction' difference and weighting magnitude
# then multiplly with speed to calculate the 'Attack Vector' --> 's_dir'
df['dir_diff'] = abs(((df['dir'] - df['angle_to_bc'])+180) % 360 - 180)
df['dir_mod'] = np.cos(np.deg2rad(df['dir_diff']))
df.loc[df['dir_mod']<0,'dir_mod'] = 0
df.loc[df['angle_to_bc'].isna(),'dir_diff'] = 0
df.loc[df['angle_to_bc'].isna(),'dir_mod'] = 1
df['s_dir'] = df['dir_mod'].mul(df['s'])

In [ ]:
# Calculate BIG Score, min distance value needed to avoid infinities and blow-up values

min_limit = 0.1
df.loc[df['dist_to_bc']<min_limit,'dist_to_bc'] = min_limit

df['big'] = ((df['a_bc']*df['a'])/df['dist_to_bc']) * (0.5*(df['s_dir']*df['o_mod']) + 0.5)

In [ ]:
# Calculate aggregated totals course of the season for each player per play
param = 'big'

dff = df_.groupby(['gameId','playId','displayName','position','jerseyNumber'])[param].sum().reset_index(name='big_tot')
dff['big_tot_sum'] = dff.groupby(['gameId','playId'])['big_tot'].transform('sum')
dff['big_tot_min'] = dff.groupby(['gameId','playId'])['big_tot'].transform('min')
dff['big_tot_max'] = dff.groupby(['gameId','playId'])['big_tot'].transform('max')
dff = dff.merge(df.groupby(['gameId','playId','displayName','jerseyNumber'])[param].max().reset_index(name='big_max'))
dff['big_max_sum'] = dff.groupby(['gameId','playId'])['big_max'].transform('sum')
dff['big_max_min'] = dff.groupby(['gameId','playId'])['big_max'].transform('min')
dff['big_max_max'] = dff.groupby(['gameId','playId'])['big_max'].transform('max')
dff = dff.merge(df.groupby(['displayName','jerseyNumber','gameId','playId'])['tackle'].max().reset_index(name='tackle'))
dff = dff.merge(df.groupby(['displayName','jerseyNumber','gameId','playId'])['assist'].max().reset_index(name='assist'))
dff = dff.merge(df.groupby(['displayName','jerseyNumber','gameId','playId'])['both'].max().reset_index(name='both'))
dff['jerseyNumber'] = dff['jerseyNumber'].astype('Int64')
dff['big_perc'] = np.round(dff['big_tot']/dff['big_tot_sum'],3)
dff['big_max_perc'] = np.round(dff['big_max']/dff['big_max_sum'],3)
dff.reset_index(drop=True,inplace=True)

In [ ]:
# Fitler out players to include only those who played at least 100 plays

min_plays = 100
min_players = dff.groupby('displayName')['playId'].count()[lambda x: x > min_plays].keys().tolist()
dff_min = dff[dff['displayName'].isin(min_players)].copy()
dff_min['play_count'] = dff_min.groupby('displayName')['playId'].transform('size')

In [ ]:
# Calculate aggregated averages for each player per play

dff_ = pd.DataFrame()
dff_['big_final'] = dff_min.groupby(['displayName','position'])['big_tot'].sum()
dff_['play_count'] = dff_min.groupby(['displayName','position'])['play_count'].max()
dff_['big_final_avg'] = dff_['big_final']/dff_['play_count']
dff_['both_final'] = dff_min.groupby(['displayName','position'])['both'].sum()
dff_['big_max_final'] = dff_min.groupby(['displayName','position'])['big_max'].sum()
dff_['big_max_avg'] = dff_['big_max_final']/dff_['play_count']
dff_['big_perc_final'] = dff_min.groupby(['displayName','position'])['big_max'].sum()
dff_['big_perc_avg'] = dff_['big_perc_final']/dff_['play_count']
dff_.reset_index(inplace=True)
dff_.sort_values(by='big_perc_avg',ascending=False).head(22)

In [ ]:
pos1 = ['DT','DE','NT']
pos2 = ['OLB','ILB','MLB']
pos3 = ['CB','FS','SS','DB']

In [ ]:
dff_.loc[dff_['position'].isin(pos3)][['headshot','Player','position','big_perc_avg']].drop_duplicates(subset='Player').sort_values(by='big_perc_avg',ascending=False).iloc[np.r_[0:5, -5:0]].to_csv('big_perc_db.csv',index=False)

In [ ]:
# Correlation between total big scores and total tackles+assists ~ 0.87

dff_[['big_final','both_final']].corr()